In [ ]:
## CARREGA BIBLIOTECAS

import nltk, re, pprint, string, csv, gensim
#from nltk import word_tokenize, FreqDist
#from nltk.corpus import stopwords
#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim import corpora
#from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
pip install sklearn

In [ ]:
## LIMPA CSV (salvar com encoding ISO 8859-1 no Sublime, depois de remover a formatação no Bloco de Notas)

## PALAVRAS REMOVIDAS PONTUALMENTE
remover = ['fls', 'nº', 'ii', 'i','é','i','§', 'sp', 'cc', 'iv', '2º', 'fl', 'r', 'n', 'dr', 'mm', 'c', 'iii']

## TRANSFORMAÇÃO DO ARQUIVO
with open("homicidio_rotulado",'r') as inFile, open("homicidio_rotulado_limpo",'w') as outFile:
    for line in inFile.readlines():
        print(" ".join([word for word in line.lower().translate(str.maketrans('', '', string.punctuation)).split() 
            if word not in stopwords.words('portuguese') and word not in remover]), file=outFile)

In [ ]:
# ACHA PALAVRAS MAIS COMUNS

f=open('homicidio_limpo','r')
raw=f.read()
tokens = nltk.word_tokenize(raw)
text = nltk.Text(tokens)
fdist = FreqDist(text)
fdist.most_common(250)
len(fdist)

In [ ]:
## CARREGA ARQUIVO - SEM STEMMING

with open("homicidio_limpo", newline="\n") as text:
    reader = csv.reader(text, delimiter=" ")
    texto = []
    for i in reader:
        texto.append(i)

In [ ]:
## CARREGA ARQUIVO NO NLTK

import string, csv, nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter

def listToString(s):  
    # initialize an empty string 
    str1 = ""  
    # traverse in the string   
    for ele in s:  
        str1 += ele
        str1 += " "
    # return string   
    return str1  

with open("homicidio_rotulado", newline="\n") as text:
    reader = csv.reader(text, delimiter=",")
    texto = []
    textopuro = []
    for i in reader:
        # split into words
        tokens = nltk.word_tokenize(str(i))
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # filter out stop words        
        stop_words = set(stopwords.words('portuguese'))
        words = [w for w in words if not w in stop_words]
        # stemming
        #porter = PorterStemmer()
        #stemmed = [porter.stem(word) for word in tokens]
        texto.append(nltk.Text(words))
        #textopuro.append(listToString(words))
        textopuro.append(str(words))
        #texto.append(nltk.Text(stemmed))

In [ ]:
## USANDO BOW

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


##vectorizer = CountVectorizer(min_df=0, lowercase=False)

## especificando bigramas
vectorizer = CountVectorizer(min_df=0, lowercase=False, ngram_range=(2,2))

vectorizer.fit(textopuro)
vector = vectorizer.transform(textopuro).toarray()

print(len(vector))
print(np.sum(vector[1]))

y_values = []

with open("desfechos.csv", newline="\n") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for i in reader: # each row is a list
        #print(i)
        y_values.append(i)

## retirar caractere estranho na primeira linha
y_values = y_values[1:len(y_values)]      

## criar conjunto de treinamento e teste
##sentences_train, sentences_test, y_train, y_test = train_test_split(vector, y_values, test_size=0.25, random_state=1000)
sentences_train, sentences_test, y_train, y_test = train_test_split(textopuro, y_values, test_size=0.25, random_state=1000)
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
print(X_train.shape)
X_test  = vectorizer.transform(sentences_test)
print(X_test.shape)

## regressão logística
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

auto_weighted_prediction = classifier.predict(X_test)
print ('Accuracy:', accuracy_score(y_test, auto_weighted_prediction))
print ('F1 score:', f1_score(y_test, auto_weighted_prediction,
                            average='weighted'))

print ('Recall:', recall_score(y_test, auto_weighted_prediction,
                              average='weighted'))

print ('Precision:', precision_score(y_test, auto_weighted_prediction,
                                    average='weighted'))

print ('\n clasification report:\n', classification_report(y_test,auto_weighted_prediction))

print ('\n confussion matrix:\n',confusion_matrix(y_test, auto_weighted_prediction))

In [ ]:
## USANDO TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

##tfidf_vectorizer=TfidfVectorizer(use_idf=True)

##especificando bigramas - ngram_range(começo,fim)
tfidf_vectorizer = TfidfVectorizer(min_df=3,max_features=None,analyzer='word',
                             token_pattern=r'\w{1,}',ngram_range=(2,2),
                             use_idf=1,smooth_idf=1)

tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(textopuro)
print(tfidf_vectorizer_vectors.shape)

# get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

y_values = []

with open("desfechos.csv", newline="\n") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for i in reader: # each row is a list
        #print(i)
        y_values.append(i)

## retirar caractere estranho na primeira linha
y_values = y_values[1:len(y_values)]    

## criar conjunto de treinamento e teste
##sentences_train, sentences_test, y_train, y_test = train_test_split(vector, y_values, test_size=0.25, random_state=1000)
sentences_train, sentences_test, y_train, y_test = train_test_split(textopuro, y_values, test_size=0.25, random_state=1000)
vectorizer = TfidfVectorizer(use_idf=True)
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
print(X_train.shape)
X_test  = vectorizer.transform(sentences_test)
print(X_test.shape)

## regressão logística
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)
auto_weighted_prediction = classifier.predict(X_test)
print ('Accuracy:', accuracy_score(y_test, auto_weighted_prediction))
print ('F1 score:', f1_score(y_test, auto_weighted_prediction,
                            average='weighted'))

print ('Recall:', recall_score(y_test, auto_weighted_prediction,
                              average='weighted'))

print ('Precision:', precision_score(y_test, auto_weighted_prediction,
                                    average='weighted'))

print ('\n clasification report:\n', classification_report(y_test,auto_weighted_prediction))

print ('\n confussion matrix:\n',confusion_matrix(y_test, auto_weighted_prediction))

In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import datapath

#model = gensim.models.KeyedVectors.load_word2vec_format('skip_s600.txt', binary=False)

model = Word2Vec.load("embeddings_word2vec_skip_s600.zip")

from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(‘model.txt’)

#wv_from_text = gensim.models.KeyedVectors.load_word2vec_format(datapath('skip_s600.txt'), binary=False)  # C text format
#wv_from_bin = gensim.models.KeyedVectors.load_word2vec_format(datapath("skip_s600.txt"), binary=True)  # C bin format

In [ ]:
## IMPORTANDO PELO MÉTODO DO ICMC

from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('skip_s600.txt')
##model = KeyedVectors.load_word2vec_format('skip_s1000.txt')

In [ ]:
## USANDO POS

import csv
import numpy

## carrega atributos
with open("homicidios - atributos.csv", newline="\n") as csvfile:
    x_values = []
    reader = csv.reader(csvfile, delimiter=';') # change contents to floats
    for i in reader: # each row is a list
        #print(i)
        x_values.append(i)

## retirar caractere estranho na primeira linha
x_values = x_values[1:len(x_values)]   
x_values = [int(i) for i in x_values] 

## carrega desfechos
with open("desfechos.csv", newline="\n") as csvfile:
    y_values = []
    reader = csv.reader(csvfile) # change contents to floats
    for i in reader: # each row is a list
        #print(i)
        y_values.append(i)

## retirar caractere estranho na primeira linha
y_values = y_values[1:len(y_values)]  
y_values = [int(i) for i in y_values] 

## criar conjunto de treinamento e teste
##sentences_train, sentences_test, y_train, y_test = train_test_split(vector, y_values, test_size=0.25, random_state=1000)
sentences_train, sentences_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.25, random_state=1000)

## regressão logística
classifier = LogisticRegression()
classifier.fit(sentences_train, y_train)
score = classifier.score(sentences_test, y_test)
print("Accuracy:", score)

In [ ]:
## OPERAÇÕES COM NLTK

#print(words[:100])
#print(texto[0])
#texto[0].similar('estado')
#texto[0].concordance('estado')

#count tokens
print(len(texto[0]))
#count exclusive words
fd = nltk.FreqDist(words)
print(len(fd))
#most common word
fd.max()

#tagging in python
#tags = nltk.pos_tag(texto[0])
#print("Parts of speech: ", tags)
#counts = Counter( tag for word,  tag in tags)
#print(counts)
#for el, cnt in counts.items():
#    print(el, '{0:2.2f}%'.format((100.0* cnt)/sbase))

tags = []

for i in range(len(texto)):
    tags = nltk.pos_tag(texto[i])
    counts = Counter( tag for word,  tag in tags)
    tags.append(counts)
    print(counts)
    
#print(type(counts))
#print(type(tags))
#print(type(Counter(tags[0])))

In [ ]:
## MAPEANDO AS TAGS PARA UM CSV

tags = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS',
        'PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN',
        'VBP','VBZ','WDT','WP','WP$','WRB']

def listToString(s):  
    # initialize an empty string 
    str1 = ""  
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    # return string   
    return str1  
    
def findCount (substring, fullstring):
    if substring in fullstring:
        pos = fullstring.find(substring)
        #print(fullstring[pos+4:pos+9])
        #print(onlyNumerics(fullstring[pos+4:pos+9]))        
        return (onlyNumerics(fullstring[pos+4:pos+9]))
    else:
        #print (0)
        return (0)

def onlyNumerics(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))

with open('pos-tags.csv', mode='w') as outfile:
    writer = csv.writer(outfile, delimiter=',')
    with open("tags", newline="\n") as text:
        reader = csv.reader(text, delimiter=" ")
        listOfTags = []
        for i in reader:
            sentence = listToString(i)
            for y in tags:
                listOfTags.append(findCount(y,sentence))
            writer.writerow(listOfTags)
            listOfTags = []

In [ ]:
## IMPORTANDO SPACY

import spacy, csv
from nltk.corpus import stopwords
from collections import Counter
nlp = spacy.load('pt_core_news_sm')
## conda install -c conda-forge spacy-model-pt_core_news_sm

In [ ]:
## OPERAÇÕES COM SPACY

doc = nlp("Tribunal de Justiça de São Paulo")
print(type(doc))

#for token in doc:
#    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#            token.shape_, token.is_alpha, token.is_stop)
    
for token in doc:
    print (token.text, token.pos_, token.pos)

''' 
with open("homicidio_limpo", newline="\n") as text:
    reader = csv.reader(text, delimiter=" ")
    texto = []
    for i in reader:
        texto.append(i)
        
print(type(texto[0]))
'''

'''
## via nltk
with open("homicidio_limpo", newline="\n") as text:
    reader = csv.reader(text, delimiter=" ")
    texto = []
    for i in reader:
        # split into words
        tokens = nltk.word_tokenize(str(i))
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # filter out stop words        
        stop_words = set(stopwords.words('portuguese'))
        words = [w for w in words if not w in stop_words]
        # stemming
        #porter = PorterStemmer()
        #stemmed = [porter.stem(word) for word in tokens]
        texto.append(nltk.Text(words))
        #texto.append(nltk.Text(stemmed))
'''  

'''
## via spacy
with open("corrupcao_limpo_sample", newline="\n") as text:
    reader = csv.reader(text, delimiter=" ")
    texto = []
    for i in reader:
        nlp = spacy.load('pt_core_news_sm')
        # split into words
        sentence = nlp(str(i))
        tokens = [token.text for token in sentence]
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # filter out stop words        
        stop_words = set(stopwords.words('portuguese'))
        words = [w for w in words if not w in stop_words]
        # stemming
        #porter = PorterStemmer()
        #stemmed = [porter.stem(word) for word in tokens]
        texto.append(nlp(str((words))))
        #texto.append(nltk.Text(stemmed))
'''

## via spacy, using NLTK, with just a sample of the text
with open("corrupcao_limpo_sample", newline="\n") as text:
    reader = csv.reader(text, delimiter=" ")
    texto = []
    for i in reader:
        # split into words
        tokens = nltk.word_tokenize(str(i))
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # filter out stop words        
        stop_words = set(stopwords.words('portuguese'))
        words = [w for w in words if not w in stop_words]
        # stemming
        #porter = PorterStemmer()
        #stemmed = [porter.stem(word) for word in tokens]
        texto.append(nlp(str(words)))
        #texto.append(nltk.Text(stemmed))
            
print(type(texto))
print(type(texto[0]))
print(len(texto))
print(len(texto[0]))

print(texto)

#print Counter (token.pos_ for token in texto[0])
c = Counter(([token.pos_ for token in texto[0]]))
print (c)
sbase = sum(c.values())
for el, cnt in c.items():
    print(el, '{0:2.2f}%'.format((100.0* cnt)/sbase))
    
#for token in nlp(str(texto[0])):
#    print (token.text, token.pos_, token.pos)

In [ ]:
print(texto[0][1])

In [ ]:
## CARREGA ARQUIVO - COM STEMMING

stemmer = nltk.stem.RSLPStemmer()

with open("homicidio_limpo", newline="\n") as texto:
    reader = csv.reader(texto, delimiter=" ")
    texto = []
    line = []
        
    for i in reader:
        for j in i:
            word = stemmer.stem(j)
            line.append(word)
        texto.append(line)
        line = []

print(texto[0])

In [ ]:
## EXEMPLO DE STEMMING EM BLOCO ÚNICO

reader = ['provas', 'produzidas', 'acusação', 'restringemse', 'apenas', 'palavras', 'policiais', 'militares', 'responsáveis']
print(type(reader))

texto = []
for i in reader:
    word = stemmer.stem(str(i))
    texto.append(word)

print(len(texto))
print(texto)

stemmer = nltk.stem.RSLPStemmer()
stemmer.stem("criminalmente")

In [ ]:
# CRIA O DICIONÁRIO DE TERMOS DO CORPUS, EM QUE CADA TERMO ÚNICO GANHA UM INDEX 
dicionario = corpora.Dictionary(texto)

# CONVERTE O DICIONÁRIO EM UMA MATRIZ DE TERMOS 
doc_matrix_termos = [dicionario.doc2bow(doc) for doc in texto]

print(len(doc_matrix_termos))
print(len(dicionario))

In [ ]:
# CRIA O LSI USANDO A BIBLIOTECA DO GENSIM

lsimodel = LsiModel(corpus=doc_matrix_termos, num_topics=10, id2word=dicionario)
lsimodel.show_topics(num_topics=5)  # Showing only the top 5 topics

In [ ]:
lsitopics = lsimodel.show_topics(formatted=False)

In [ ]:
# MODELAGEM DE TÓPICOS VIA HDP

hdpmodel = HdpModel(corpus=doc_matrix_termos, id2word=dicionario)
hdpmodel.show_topics()

In [ ]:
hdptopics = hdpmodel.show_topics(formatted=False)
hdpmodel.save("hdp_homicidio_com_stem")

In [ ]:
# CRIA O LDA USANDO A BIBLIOTECA DO GENSIM
lda = gensim.models.ldamodel.LdaModel

# RODA E TREINA O LDA NO DICIONARIO DE TERMOS E NA MATRIZ
ldamodel = lda(doc_matrix_termos, num_topics=10, id2word = dicionario, passes=50)
ldamodel.save("lda_homicidio_com_stem")

In [ ]:
# CARREGA MODELO EXISTENTE DE LDA

#lda = gensim.models.ldamodel.LdaModel

# RODA E TREINA O LDA NO DICIONARIO DE TERMOS E NA MATRIZ
#lda.load("lda_esquerda")

ldamodel = gensim.models.LdaModel.load('lda_corrupcao1')

In [ ]:
## MOSTRA TÓPICOS DO LDA

ldamodel.show_topics(num_topics=10, num_words=20, log=False, formatted=True)

In [ ]:
## IMPRIME O RESULTADO DO LDA COM NUM_TOPICS DE TÓPICOS

print(ldamodel.print_topics(num_topics=10, num_words=5))

In [ ]:
ldatopics = ldamodel.show_topics(formatted=False)

In [ ]:
## PLOTA O MODELO DO LDA

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(ldamodel, doc_matrix_termos, dicionario)

In [ ]:
## DEFINE NÚMERO ÓTIMO DE TÓPICOS DO LDA

def evaluate_graph(dictionary, corpus, texts, limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    for num_topics in range(1, limit):
        lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
        
    # Show graph
    x = range(1, limit)
    plt.plot(x, c_v)
    plt.xlabel("num_topics")
    plt.ylabel("Coherence score")
    plt.legend(("c_v"), loc='best')
    plt.show()
    
    return lm_list, c_v

In [ ]:
## RODA O ALGORITMO ACIMA

lmlist, c_v = evaluate_graph(dictionary=dicionario, corpus=doc_matrix_termos, texts=texto, limit=10)

In [ ]:
lmtopics = lmlist[5].show_topics(formatted=False)

In [ ]:
## PLOTA A AVALIAÇÃO DE TÓPICOS DO LDA

pyLDAvis.gensim.prepare(lmlist[2], doc_matrix_termos, dicionario)

In [ ]:
## ENCONTRA OS MELHORES TÓPICOS DO LDA

def ret_top_model():
    """
    Since LDAmodel is a probabilistic model, it comes up different topics each time we run it. To control the
    quality of the topic model we produce, we can see what the interpretability of the best topic is and keep
    evaluating the topic model until this threshold is crossed. 
    
    Returns:
    -------
    lm: Final evaluated topic model
    top_topics: ranked topics in decreasing order. List of tuples
    """
    top_topics = [(0, 0)]
    while top_topics[0][1] < 0.97:
        lm = LdaModel(corpus=doc_matrix_termos, id2word=dicionario)
        coherence_values = {}
        for n, topic in lm.show_topics(num_topics=-1, formatted=False):
            topic = [word for word, _ in topic]
            cm = CoherenceModel(topics=[topic], texts=texto, dictionary=dicionario, window_size=10)
            coherence_values[n] = cm.get_coherence()
        top_topics = sorted(coherence_values.items(), key=operator.itemgetter(1), reverse=True)
    return lm, top_topics

In [ ]:
## RODA O ALGORITMO ACIMA

lm, top_topics = ret_top_model()
print(top_topics[:5])

In [ ]:
## AVALIA TODOS OS MÉTODOS ESTATÍSTICOS 

lsitopics = [[word for word, prob in topic] for topicid, topic in lsitopics]

hdptopics = [[word for word, prob in topic] for topicid, topic in hdptopics]

ldatopics = [[word for word, prob in topic] for topicid, topic in ldatopics]

lmtopics = [[word for word, prob in topic] for topicid, topic in lmtopics]

lsi_coherence = CoherenceModel(topics=lsitopics[:10], texts=texto, dictionary=dicionario, window_size=10).get_coherence()

hdp_coherence = CoherenceModel(topics=hdptopics[:10], texts=texto, dictionary=dicionario, window_size=10).get_coherence()

lda_coherence = CoherenceModel(topics=ldatopics, texts=texto, dictionary=dicionario, window_size=10).get_coherence()

lm_coherence = CoherenceModel(topics=lmtopics, texts=texto, dictionary=dicionario, window_size=10).get_coherence()

#lda_lsi_coherence = CoherenceModel(topics=lda_lsi_topics[:10], texts=texto, dictionary=dicionario, window_size=10).get_coherence()

In [ ]:
## PLOTA O GRÁFICO DE AVALIAÇÃO

def evaluate_bar_graph(coherences, indices):
    """
    Function to plot bar graph.
    
    coherences: list of coherence values
    indices: Indices to be used to mark bars. Length of this and coherences should be equal.
    """
    assert len(coherences) == len(indices)
    n = len(coherences)
    x = np.arange(n)
    plt.bar(x, coherences, width=0.2, tick_label=indices, align='center')
    plt.xlabel('Models')
    plt.ylabel('Coherence Value')
    
    
#evaluate_bar_graph([lsi_coherence, hdp_coherence, lda_coherence, lm_coherence, lda_lsi_coherence],
#                   ['LSI', 'HDP', 'LDA', 'LDA_Mod', 'LDA_LSI'])

evaluate_bar_graph([lsi_coherence, hdp_coherence, lda_coherence, lm_coherence],
                   ['LSI', 'HDP', 'LDA', 'LDA_Mod'])